<a href="https://colab.research.google.com/github/carlibeisel/Drains_Lower_Boise_River/blob/main/_UBRB_precip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Created by Carli Beisel

Purpose: This script extracts precipitation data from Google Earth Engine using the Earth Engine API in Python for the Upper Boise River Basin.



## Import packages/libraries and connect to Google drive

In [ ]:
# Installs geemap package
import subprocess
!pip install geemap

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project = 'extract-gridmet') #input and create project on google earth engiine

In [ ]:
!pip install geopandas
import geopandas as gpd
import json
import os
import numpy as np
import pandas as pd
!pip install pycrs
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## UBRB shapefile

## Annual Precipitation for UBRB

In [ ]:
## ------------------------------------------------- ##
##          ANNUAL PRECIPITATION METRIC              ##
## ------------------------------------------------- ##
ubrb = emap.shp_to_ee('/content/drive/MyDrive/Data/GIS Shapefiles/UBRB_shp/UBRB.shp')

years = np.arange(1987,2021)
annual_pr = []

for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-01-01'), (str(years[i])+'-12-31')) #get image collection for entire year
  pr = daymet.select('prcp').map(lambda image: image.clip(ubrb)).sum().set({'system:index': str(years[i])}) #select precip to analyze and sum
  annual_pr.append(pr)

annual_pr = ee.ImageCollection(annual_pr)


out_stats = os.path.join('/content/drive/MyDrive/Data/Model Modifications/UBRB_precip/ubrb_prcp.csv')
emap.zonal_statistics(annual_pr, ubrb, out_stats, statistics_type='MEAN', scale=1000)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Data/Model Modifications/UBRB_precip/ubrb_prcp.csv


In [ ]:
#---------------------#
#        Clean        #
#---------------------#

#reorganize csv file to make manageable
data = pd.read_csv('/content/drive/MyDrive/Data/Model Modifications/UBRB_precip/ubrb_prcp.csv')

df = pd.DataFrame(data)
df = df.T #transpose df

# Set the first row as the column names
df.columns = df.iloc[0]
df = df[0:]
df = df.drop(df.index[34:]) #drop the last few lines

# Reset the index and rename the columns
df = df.reset_index()
df.columns = ['year', 'precip']

# Remove '_prcp' from the 'year' column
df['year'] = df['year'].str.replace('_prcp', '')

print(df)

#save
df.to_csv('/content/drive/MyDrive/Data/Model Modifications/UBRB_precip/ubrb_prcp.csv', index=False)



    year       precip
0   1987   537.560081
1   1988   734.115144
2   1989    630.99135
3   1990   715.458645
4   1991   720.412696
5   1992   572.140523
6   1993   783.622227
7   1994   643.050725
8   1995   1066.30204
9   1996  1245.321383
10  1997   712.542221
11  1998   1041.10296
12  1999   711.782978
13  2000   751.041632
14  2001    642.21541
15  2002   568.921902
16  2003    766.13772
17  2004   688.722037
18  2005   833.791812
19  2006     908.4052
20  2007   629.139131
21  2008   766.231906
22  2009   739.163377
23  2010   913.614161
24  2011   829.458274
25  2012   910.976358
26  2013   500.880187
27  2014   963.203312
28  2015   776.144241
29  2016   724.308265
30  2017  1136.112104
31  2018   672.778805
32  2019   860.305032
33  2020   733.291005


In [ ]:
#---------------------#
#        Scale        #
#---------------------#
mean_precip = df['precip'].mean()
std_dev_precip = df['precip'].std()
df['precip_scaled'] = (df['precip'] - mean_precip) / std_dev_precip ** 2

df = df.reset_index(drop=True)
df.to_csv('/content/drive/MyDrive/Data/Model Modifications/UBRB_precip/ubrb_prcp_scaled.csv', index=False)

In [ ]:
# not needed?
## ---------------------------------------------------------------------- ##
##   IMPORT THE DAYMET DATA FOR PRECIPITATION PRIOR TO IRRIGATION SEASON  ##
## ---------------------------------------------------------------------- ##

years = np.arange(1987,2021)
ant_pr = []
for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate(str(years[i] - 1) + '-01-01', str(years[i] - 1) + '-12-31') #trying to get full year
  date = str(years[i]+1)
  print(date)
  prcp = daymet.select('prcp').map(lambda image: image.clip(ubrb)).sum().set({'system:index':date}) #select the bands to analyze
  ant_pr.append(prcp) #calculate the mean across all pixels

ant_precip = ee.ImageCollection(ant_pr) #convert list of image to image collection for zonal stats command

precip_vis = {
  'min': 0,
  'max': 544,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(ant_precip, precip_vis, 'prcp')
Map

1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


Map(center=[43.615, -116.2023], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

##**LBRB precip** : For reference

In [ ]:
# Load the shapefile
lbrb = emap.shp_to_ee('/content/drive/MyDrive/Data/GIS Shapefiles/LBRB_shp/LBRB_file.shp') # converts shapefile to feature in GEE


years = np.arange(1987,2021)
annual_pr = []

for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-01-01'), (str(years[i])+'-12-31')) #get image collection for entire year
  pr = daymet.select('prcp').map(lambda image: image.clip(lbrb)).sum().set({'system:index': str(years[i])}) #select precip to analyze and sum
  annual_pr.append(pr)

annual_pr = ee.ImageCollection(annual_pr)


out_stats = os.path.join('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/UBRB_precip/lbrb_prcp.csv')
emap.zonal_statistics(annual_pr, lbrb, out_stats, statistics_type='MEAN', scale=1000)

Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/UBRB_precip/lbrb_prcp.csv
